# Limpieza de datos

In this notebook the csv air pollution data is cleaned. Said data was obtained from https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default.

In [2]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import datetime

In [3]:
df = pd.read_csv('../original_datasets/ene_mo20.csv', sep=';')

In [4]:
df.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2020,1,1,7.0,V,...,12.0,V,14.0,V,12.0,V,11.0,V,9.0,V
1,28,79,4,1,28079004_1_38,2020,1,2,8.0,V,...,14.0,V,14.0,V,15.0,V,12.0,V,10.0,V
2,28,79,4,1,28079004_1_38,2020,1,3,9.0,V,...,12.0,V,13.0,V,13.0,V,10.0,V,8.0,V
3,28,79,4,1,28079004_1_38,2020,1,4,7.0,V,...,10.0,V,11.0,V,10.0,V,8.0,V,9.0,V
4,28,79,4,1,28079004_1_38,2020,1,5,8.0,V,...,12.0,V,14.0,V,13.0,V,11.0,V,9.0,V


The explanation for each column can be found in the file Instrucciones-datasets-Madrid.pdf. In the following I'll prove that the ESTACION column is the same as the last two digits in the station code section of PUNTO_MUESTREO.

In [5]:
df['ESTACION'].unique()

array([ 4,  8, 11, 16, 17, 18, 24, 27, 35, 36, 38, 39, 40, 47, 48, 49, 50,
       54, 55, 56, 57, 58, 59, 60])

In [6]:
pmuestreo = df['PUNTO_MUESTREO'].str.extract(r'^(\d*)_(\d*)_(\d*)$') #Obtiene una columna por seccion

In [7]:
pmuestreo.head()

,0,1,2
0,28079004,1,38
1,28079004,1,38
2,28079004,1,38
3,28079004,1,38
4,28079004,1,38


In [8]:
estaciones = pmuestreo[0].str.extract(r'^280790(\d*)$').astype(int) #Obtiene solo la ultima parte

In [9]:
dif_est = estaciones.values - df['ESTACION'].values.reshape(-1,1)
np.where(dif_est != 0)

(array([], dtype=int64), array([], dtype=int64))

In [10]:
np.where(dif_est != 0)

(array([], dtype=int64), array([], dtype=int64))

So the PUNTO_MUESTREO column, in addition to PROVINCIA and MUNICIPIO, aren't useful and are thus removed.

In [11]:
df.drop(columns = ['PUNTO_MUESTREO', 'PROVINCIA', 'MUNICIPIO'], inplace = True)

Renaming columns from uppercase to lowercase.

In [12]:
df.columns = df.columns.str.lower()

In [13]:
df.head()

,estacion,magnitud,ano,mes,dia,h01,v01,h02,v02,h03,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
0,4,1,2020,1,1,7.0,V,8.0,V,9.0,...,12.0,V,14.0,V,12.0,V,11.0,V,9.0,V
1,4,1,2020,1,2,8.0,V,8.0,V,7.0,...,14.0,V,14.0,V,15.0,V,12.0,V,10.0,V
2,4,1,2020,1,3,9.0,V,8.0,V,7.0,...,12.0,V,13.0,V,13.0,V,10.0,V,8.0,V
3,4,1,2020,1,4,7.0,V,5.0,V,6.0,...,10.0,V,11.0,V,10.0,V,8.0,V,9.0,V
4,4,1,2020,1,5,8.0,V,6.0,V,5.0,...,12.0,V,14.0,V,13.0,V,11.0,V,9.0,V


The v- columns indicate if the data is validated per hour. I'll check how many aren't verified.

In [14]:
hours = ['0' + str(i) if i<10 else str(i) for i in range(1,25)]

In [15]:
unval = []
for hour in hours:
    unval_hour_rows = df[df['v' + hour] == 'N']
    unval.append(unval_hour_rows)
unval_df = pd.concat(unval)
unval_df.drop_duplicates(inplace = True)
num_unval = len(unval_df)

In [16]:
#Porcentaje de datos no verificados
num_rows = len(df)
num_unval/num_rows

0.08480715959940337

Approximately 8.5% of the data aren't verified. I'll ignore this for the moment.

In [17]:
df.drop(columns = ['v0'+str(i) if i<10 else 'v'+str(i) for i in range(1,25)], inplace = True)

Creating a date column.

In [18]:
anios = df['ano'].astype('str')
meses = df['mes'].astype('str')
dias = df['dia'].astype('str')

fechas_str = anios + '-' + meses + '-' + dias

df['fecha'] = pd.to_datetime(fechas_str, yearfirst=True)

df.drop(columns=['ano','mes','dia'], inplace=True)

Creating a column with the name of each pollutant that will be used.

In [19]:
df['magnitud'].unique()

array([ 1,  6,  7,  8, 12,  9, 10, 14, 20, 30, 35, 42, 43, 44])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4693 entries, 0 to 4692
Data columns (total 27 columns):
estacion    4693 non-null int64
magnitud    4693 non-null int64
h01         4693 non-null float64
h02         4693 non-null float64
h03         4693 non-null float64
h04         4693 non-null float64
h05         4693 non-null float64
h06         4693 non-null float64
h07         4693 non-null float64
h08         4693 non-null float64
h09         4693 non-null float64
h10         4693 non-null float64
h11         4693 non-null float64
h12         4693 non-null float64
h13         4693 non-null float64
h14         4693 non-null float64
h15         4693 non-null float64
h16         4693 non-null float64
h17         4693 non-null float64
h18         4693 non-null float64
h19         4693 non-null float64
h20         4693 non-null float64
h21         4693 non-null float64
h22         4693 non-null float64
h23         4693 non-null float64
h24         4693 non-null float64
fecha       4

In [21]:
#From the above it can be seen that the df does not contain any missing values, so the
#dropna I call below will only affect what it's meant to affect.
used = {1:'SO2',
        6:'CO',
        8:'NO2',
        9:'PM2.5',
        10:'PM10',
        14:'O3'
        }

not_used = {i:np.nan for i in (7,12,20,30,35,42,43,44)}
pollutants_dict = {**used, **not_used}

df['contaminante'] = df['magnitud'].apply(lambda x:pollutants_dict[x])

df = df.dropna().reset_index(drop=True)

df.drop(columns=['magnitud'], inplace = True)

In [22]:
df.head()

,estacion,h01,h02,h03,h04,h05,h06,h07,h08,h09,...,h17,h18,h19,h20,h21,h22,h23,h24,fecha,contaminante
0,4,7.0,8.0,9.0,8.0,6.0,6.0,5.0,5.0,4.0,...,11.0,10.0,10.0,12.0,14.0,12.0,11.0,9.0,2020-01-01,SO2
1,4,8.0,8.0,7.0,6.0,5.0,5.0,5.0,9.0,10.0,...,11.0,10.0,11.0,14.0,14.0,15.0,12.0,10.0,2020-01-02,SO2
2,4,9.0,8.0,7.0,7.0,6.0,6.0,6.0,9.0,9.0,...,11.0,12.0,12.0,12.0,13.0,13.0,10.0,8.0,2020-01-03,SO2
3,4,7.0,5.0,6.0,5.0,4.0,4.0,4.0,4.0,4.0,...,9.0,9.0,10.0,10.0,11.0,10.0,8.0,9.0,2020-01-04,SO2
4,4,8.0,6.0,5.0,4.0,3.0,3.0,3.0,3.0,4.0,...,11.0,11.0,11.0,12.0,14.0,13.0,11.0,9.0,2020-01-05,SO2


The most complicated part is the following: eliminating all the independent hourly measurement columns and subsituting them for a single column with the pollutant concentrations at a given datetime, given by a different column.

In [23]:
df.reindex(np.sort(df.columns), axis = 1).columns

Index(['contaminante', 'estacion', 'fecha', 'h01', 'h02', 'h03', 'h04', 'h05',
       'h06', 'h07', 'h08', 'h09', 'h10', 'h11', 'h12', 'h13', 'h14', 'h15',
       'h16', 'h17', 'h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24'],
      dtype='object')

In [24]:
def columns_to_datetime(df):
    """Creates a new df with a datetime index and values corresponding to the concentration of a pollutant
    at said datetime.
    Arguments:
    df: A dataframe with different columns for pollutant concentration per hour of the day, named
    in the format h01, h02, etc."""
    
    hours = ('h0'+str(i) if i<10 else 'h'+str(i) for i in range(1,25))
    df_hours = []
    df = df.reindex(np.sort(df.columns), axis = 1) #To prevent issues coming from columns
    #being in a different order when I pass a list of columns later on
    
    for hour, hour_str in enumerate(hours):
        hour = hour + 1
        
        #Creating a df with only the values of the corresponding hour
        columns = ['contaminante', 'estacion', 'fecha', hour_str]
        df_hour = df[columns]
        
        #The hour is appended to the date
        df_hour['dt'] = df_hour['fecha'].apply(lambda x:datetime.datetime(x.year, x.month, x.day, hour%24))
        df_hour.drop(columns=['fecha'], inplace=True)
        
        #Changing the name of the concentration column to avoid problems
        df_hour.rename(columns={hour_str:'concentracion'}, inplace = True)
        
        #The df is appended to the list
        df_hours.append(df_hour)
    
    new_df = pd.concat(df_hours).sort_values('dt').reset_index(drop=True)
    assert len(new_df) == 24*len(df)
    
    return new_df

In [25]:
new_df = columns_to_datetime(df)

/home/yamil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/yamil/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/yamil/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retu

In [26]:
new_df.head()

,contaminante,estacion,concentracion,dt
0,NO2,36,83.00,2020-01-01
1,NO2,54,105.00,2020-01-01
2,O3,17,2.04,2020-01-01
3,CO,4,0.70,2020-01-01
4,SO2,57,9.00,2020-01-01


Creating different dataframes for each pollutant.

In [27]:
dfs_pollutant = []
for pollutant in df['contaminante'].unique():
    df2 = new_df[new_df['contaminante'] == pollutant].copy()
    df2.drop(columns=['contaminante'], inplace=True)

In [28]:
new_df['dt'].dt.year.unique().astype(str)

array(['2020'], dtype='<U21')

In [29]:
NO2 = new_df[new_df['contaminante'] == 'NO2'].copy()

In [30]:
NO2.drop(columns=['contaminante'],inplace=True)

In [31]:
NO2.head()

,estacion,concentracion,dt
0,36,83.0,2020-01-01
1,54,105.0,2020-01-01
5,56,67.0,2020-01-01
8,4,61.0,2020-01-01
14,18,61.0,2020-01-01


In [32]:
NO2.set_index('dt')

,estacion,concentracion
dt,,
2020-01-01 00:00:00,36,83.0
2020-01-01 00:00:00,54,105.0
2020-01-01 00:00:00,56,67.0
2020-01-01 00:00:00,4,61.0
2020-01-01 00:00:00,18,61.0
...,...,...
2020-01-31 23:00:00,4,42.0
2020-01-31 23:00:00,36,56.0
2020-01-31 23:00:00,18,46.0


The station located at Avda. La Guardia (station code 28079054) is set outside the urban environment of the city and could therefore be registering outlier values. It will be removed.

In [33]:
LaGuardia = df[df['estacion'] == 54].index
df.drop(index=LaGuardia).reset_index(drop=True)

,estacion,h01,h02,h03,h04,h05,h06,h07,h08,h09,...,h17,h18,h19,h20,h21,h22,h23,h24,fecha,contaminante
0,4,7.00,8.00,9.00,8.00,6.00,6.00,5.00,5.00,4.00,...,11.00,10.00,10.00,12.00,14.00,12.00,11.00,9.00,2020-01-01,SO2
1,4,8.00,8.00,7.00,6.00,5.00,5.00,5.00,9.00,10.00,...,11.00,10.00,11.00,14.00,14.00,15.00,12.00,10.00,2020-01-02,SO2
2,4,9.00,8.00,7.00,7.00,6.00,6.00,6.00,9.00,9.00,...,11.00,12.00,12.00,12.00,13.00,13.00,10.00,8.00,2020-01-03,SO2
3,4,7.00,5.00,6.00,5.00,4.00,4.00,4.00,4.00,4.00,...,9.00,9.00,10.00,10.00,11.00,10.00,8.00,9.00,2020-01-04,SO2
4,4,8.00,6.00,5.00,4.00,3.00,3.00,3.00,3.00,4.00,...,11.00,11.00,11.00,12.00,14.00,13.00,11.00,9.00,2020-01-05,SO2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,60,30.47,28.57,40.20,20.83,23.70,29.73,35.75,17.03,21.68,...,35.52,55.73,45.52,53.66,55.10,54.08,62.30,65.94,2020-01-27,O3
2322,60,65.97,67.20,66.48,66.08,64.78,66.00,63.62,53.22,41.99,...,58.85,54.94,51.97,45.63,45.63,49.11,54.61,58.76,2020-01-28,O3
2323,60,61.76,59.95,60.05,61.18,61.73,59.96,53.67,42.92,28.33,...,58.62,54.71,52.95,51.45,46.65,51.23,53.76,57.38,2020-01-29,O3
2324,60,58.83,60.27,60.13,60.71,62.78,62.58,56.39,45.03,35.66,...,68.03,59.34,54.16,48.83,49.70,55.40,63.69,66.70,2020-01-30,O3


In [34]:
df['estacion'].unique()

array([ 4,  8, 11, 16, 17, 18, 24, 27, 35, 36, 38, 39, 40, 47, 48, 49, 50,
       54, 55, 56, 57, 58, 59, 60])

In [37]:
new_df.set_index('dt', inplace=True)

In [49]:
new_df[new_df.index.day == 15]

,contaminante,estacion,concentracion
dt,,,
2020-01-15 00:00:00,CO,16,0.50
2020-01-15 00:00:00,PM10,24,35.00
2020-01-15 00:00:00,NO2,59,57.00
2020-01-15 00:00:00,O3,39,2.36
2020-01-15 00:00:00,O3,35,4.73
...,...,...,...
2020-01-15 23:00:00,PM10,55,1.00
2020-01-15 23:00:00,PM2.5,8,24.00
2020-01-15 23:00:00,NO2,11,98.00


In [47]:
NO2.set_index('dt',inplace=True)

In [48]:
NO2.index

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 00:00:00',
               '2020-01-01 00:00:00', '2020-01-01 00:00:00',
               '2020-01-01 00:00:00', '2020-01-01 00:00:00',
               '2020-01-01 00:00:00', '2020-01-01 00:00:00',
               '2020-01-01 00:00:00', '2020-01-01 00:00:00',
               ...
               '2020-01-31 23:00:00', '2020-01-31 23:00:00',
               '2020-01-31 23:00:00', '2020-01-31 23:00:00',
               '2020-01-31 23:00:00', '2020-01-31 23:00:00',
               '2020-01-31 23:00:00', '2020-01-31 23:00:00',
               '2020-01-31 23:00:00', '2020-01-31 23:00:00'],
              dtype='datetime64[ns]', name='dt', length=17616, freq=None)